# 버전확인

In [2]:
import sklearn
print("scikit-learn 버전:", sklearn.__version__)

scikit-learn 버전: 1.3.2


In [3]:
pip install pickle

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement pickle (from versions: none)
ERROR: No matching distribution found for pickle


In [4]:
# 수정사항 1 - konlpy 설치를 위해 자바홈 설정
import os
os.environ['JAVA_HOME'] = 'C:\Program Files\Java\jdk-17'  # 설치 경로에 맞게 수정

In [5]:
# 수정사항 2 - konlpy 설치
pip install konlpy

SyntaxError: invalid syntax (2506537345.py, line 2)

In [6]:
import konlpy

print("konlpy 버전:", konlpy.__version__)

konlpy 버전: 0.6.0


# API 구현 예제

In [12]:
# from flask import Flask, request, jsonify
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import pickle

# 수정사항3-  konlpy 임포트
from konlpy.tag import Kkma

# 수정사항4 - Kkma 인스턴스 초기화
kkma = Kkma()

# app = Flask(__name__)

# 수정사항7 - pkl파일 변경
# 저장된 전처리 데이터를 로드
with open('processed_data.pkl', 'rb') as f:
    data, tokenized_symptoms, tfidf_matrix, vectorizer = pickle.load(f)

# @app.route('/predict', methods=['POST'])
def predict(input_value_raw):
    # input_value_raw = request.json.get('symptoms')
    input_values = input_value_raw.split(",")

    # selected_symptoms_info 리스트 초기화: 선택된 증상 정보를 저장합니다.
    selected_symptoms_info = []

    # selected_symptoms_id_set 집합 초기화: 선택된 증상의 ID를 저장하여 중복을 방지합니다.
    selected_symptoms_id_set = set()

    # 수정사항5 - 형태소 분석을 통해 입력 값을 토큰화
    input_tokenized = [" ".join(kkma.morphs(input_value)) for input_value in input_values]
    
    for input_value, input_tok in zip(input_values, input_tokenized):
        for index, row in data.iterrows():
            symptom = row['symptoms']
            if symptom in input_value:
                # symptoms이 완전히 일치하는 경우에 대해 처리
                if row['symptom_ids'] not in selected_symptoms_id_set:
                    selected_symptoms_info.append((symptom, 1.0, row['symptom_ids'], row['symptom_describe']))
                    selected_symptoms_id_set.add(row['symptom_ids'])
    
        # 타겟 문장의 TF-IDF 벡터화
        target_vector = vectorizer.transform([input_tok])
    
        # 증상과 타겟 문장 간의 유사도 계산
        similarities = cosine_similarity(target_vector, tfidf_matrix)
    
        # 유사도를 리스트로 변환
        similarity_list = similarities.flatten()
        
        # 유사도 임계값 설정
        similarity_threshold = 0.35

        # 수정사항6 - 유사도 리스트를 내림차순 정렬 및 10개 추출 부분 수정 
        # 유사도 기반으로 상위 10개의 증상 선택
        top_indices = similarity_list.argsort()[-10:][::-1]  # 상위 10개의 인덱스를 유사도 기준으로 내림차순 정렬
        for idx in top_indices:
            if similarity_list[idx] > similarity_threshold:  # 유사도가 0보다 큰 경우만 처리
                symptom = data.loc[idx, 'symptoms']
                if data.loc[idx, 'symptom_ids'] not in selected_symptoms_id_set:
                    selected_symptoms_info.append((symptom, similarity_list[idx], data.loc[idx, 'symptom_ids'], data.loc[idx, 'symptom_describe']))
                    selected_symptoms_id_set.add(data.loc[idx, 'symptom_ids'])

    # 최종적으로 selected_symptoms_info에는 중복되지 않은 증상 정보가 최대 10개까지 저장됩니다.
    response = []
    for i, (symptom, similarity, symptom_id, symptom_description) in enumerate(selected_symptoms_info, 1):
        response.append({
            "symptom": symptom,
            "symptom_id": symptom_id,
            "similarity": similarity,
            "symptom_description": symptom_description
        })

    # 수정사항7 - response 내림차순 정렬 
    # similarity 값 기준으로 내림차순 정렬
    response = sorted(response, key=lambda x: x['similarity'], reverse=True)

    if not response:
        return None
    return response

# if __name__ == '__main__':
#     app.run(debug=True)


C:\Users\user\miniconda3\envs\ml2\lib\site-packages\sklearn\base.py:348: InconsistentVersionWarning: Trying to unpickle estimator TfidfTransformer from version 1.2.2 when using version 1.3.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
C:\Users\user\miniconda3\envs\ml2\lib\site-packages\sklearn\base.py:348: InconsistentVersionWarning: Trying to unpickle estimator TfidfVectorizer from version 1.2.2 when using version 1.3.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [13]:
predict("기침, 손발이 떨림")

[{'symptom': '기침',
  'symptom_id': 'SS000868',
  'similarity': 1.0,
  'symptom_description': '기도 분비물이나 외부 먼지, 미생물 등을 제거하기 위해 갑작스럽게 숨을 내뿜는 반사 행동\n'},
 {'symptom': '떨림(진전)',
  'symptom_id': 'SS000710',
  'similarity': 0.6245823203071182,
  'symptom_description': '떨림 (떨림)'},
 {'symptom': '손떨림',
  'symptom_id': 'SS000500',
  'similarity': 0.5581655495853738,
  'symptom_description': '손이 떨리는 상태\n'},
 {'symptom': '치아 흔들림',
  'symptom_id': 'SS000579',
  'similarity': 0.4754387435848986,
  'symptom_description': '이가 떨리는 소리'},
 {'symptom': '객혈',
  'symptom_id': 'SS000863',
  'similarity': 0.4247109009279538,
  'symptom_description': '혈액이 섞인 가래가 기침과 함께 나오는 것\n'},
 {'symptom': '온몸이 떨림',
  'symptom_id': 'SS000689',
  'similarity': 0.36369876122973926,
  'symptom_description': '온몸이 떨리는 증상'},
 {'symptom': '사래걸림',
  'symptom_id': 'SS000876',
  'similarity': 0.350152969197988,
  'symptom_description': '기관, 기관지, 폐에 이물질을 흡입하여 발생하는 기침 등의 호흡기 반응'}]

In [ ]:
기침, 떨림(진전), 손떨림, 치아 흔들림, 객혈, 온몸이 떨림, 사래걸림